# Working with DataFrames.jl v1.5.0

# Part 1

## Bogumił Kamiński

In this part of the tutorial we will use the *Airports2.csv* file that is available for download here: https://www.kaggle.com/flashgordon/usa-airport-dataset. Therefore before starting it please be sure to download it and and have it un-zipped in your working directory.

Our objective is to show selected features of v1.5.0 release of the DataFrames.jl package.

This tutorial is not meant to be a replacement of a documentation - it just shows some practical application examples. If you want to know all the details of the discussed functionalities please refer to docstrings of the functions we show.

As usual in Julia, for every project you should have *Project.toml* and *Manifest.toml* files that specify the dependencies. They are bundled with this file in a GitHub gist.

If you do not have much experience with project dependencies and want to understand more about managing them I recommend you to read https://bkamins.github.io/julialang/2020/05/18/project-workflow.html.

This tutorial was developed under Julia 1.9.0.

Before we start let us make sure that you have the right versions of packages installed.

The output of the command below should be:
```
  [6e4b80f9] BenchmarkTools v1.3.2
  [336ed68f] CSV v0.10.9
  [8be319e6] Chain v0.5.0
  [a93c6f00] DataFrames v1.5.0
  [7073ff75] IJulia v1.24.0
  [0f8b85d8] JSON3 v1.12.0
  [91a5bcdd] Plots v1.38.5
```

In [ ]:
] status

We start with loading the required packages and reading in the CSV file to a `DataFrame`:

In [ ]:
using BenchmarkTools
using CSV
using DataFrames
using Dates
using Chain
using Plots

In [ ]:
df = CSV.read("Airports2.csv", DataFrame, missingstring="NA")

Our data frame contains the following columns:

1. `:Origin_airport`: Three letter airport code of the origin airport
2. `:Destination_airport`: Three letter airport code of the destination airport
3. `:Origin_city`: Origin city name
4. `:Destination_city`: Destination city name
5. `:Passengers`: Number of passengers transported from origin to destination
6. `:Seats`: Number of seats available on flights from origin to destination
7. `:Flights`: Number of flights between origin and destination (multiple records for one month, many with flights > 1)
8. `:Distance`: Distance (to nearest mile) flown between origin and destination
9. `:Fly_date`: The date (yyyymm) of flight
10. `:Origin_population`: Origin city's population as reported by US Census
11. `:Destination_population`: Destination city's population as reported by US Census

Let us first investigate its metadata

In [ ]:
size(df)

In [ ]:
nrow(df)

In [ ]:
ncol(df)

In [ ]:
names(df)

In [ ]:
propertynames(df)

An important thing to note is that the `names` function returns a vector of `String`s and the `propertynames` function returns a vector of `Symbol`s.

In DataFrames.jl both strings and `Symbol`s can be used for column indexing.

Let us now get some summary statistics of our data set using the `describe` function:

In [ ]:
describe(df)

In particular note that CSV.jl correctly idenfitied `:Fly_date` column as being a `Date` type.

When we investigate the summary statistics we note that there are flights that potentially have `0` passengers, `0` seats, or `0` flights.

In practice if you get such data it is good to investigate it, as it shows some potential data quality issues.

So, let us investigate it:

In [ ]:
@chain df begin
    select(:Passengers, :Seats, :Flights)
    mapcols(ByRow(>(0)), _)  # or: mapcols(ByRow( x -> x > 0 ), _ )
    groupby(:, sort=true)
    combine(nrow, proprow)
end

Before interpreting the results let us dissect the code:
1. `@chain` is a macro from Chain.jl package, that makes it easy to chain function calls and makes `_` a placeholder for a value returned by a function that was earlier in the chain (you can read about the details here: https://github.com/jkrumbiegel/Chain.jl). Often, we don't even need to placeholder `_` - particularly in a DataFrames.jl context, where we give the result of an operation *as first argument* to the next operation.
2. `select` creates a new data frame that keeps only three columns that we are interested in
3. the `mapcols` statement applies a function to all columns. In this case, use `ByRow(>(0))`, which is basically the function `>(0)` wrapped in the `ByRow` helper function - which applies a function *to each row*. Notice that we could also have written `ByRow( x -> x > 0 )` if we found that easier to read. This line will transform each column of our data frame to have the `Bool` element type (check the output above!). On each row and for each column we will have a value `true` if an entry is greater than `0` and `false` otherwise. In this way we are able to easily idenfity rows with `0` entries, as they are most interesting for us
4. `groupby` groups our data frame by all columns (indicated by the `:` selector) - i.e. we want to group this data by lumping together all the rows where `Passengers` is `true`, `Seats` is `true`, `Flights` is `true`, and so on. Finally we tell DataFrames.jl that the gorups should be sorted.
5. finally `combine` just counts the number and proportion of rows in each group

Now going to our results - some of entries can be explained, like `0` passenges, but some seats and some flights (I guess it means that just no one took some flight).

However cases like some passengers, but `0` seats and `0` flights are probably a mistake in data (we have 7 rows that have this combination of values).

Let us try to find these 7 rows in two ways:

In [ ]:
filter(row -> row.Passengers > 0 && row.Seats == 0 && row.Flights == 0, df)

In [ ]:
subset(df, :Passengers => ByRow(>(0)), :Seats => ByRow(==(0)), :Flights  => ByRow(==(0)))

Let us compare the timing of both options:

In [ ]:
@time filter(row -> row.Passengers > 0 && row.Seats == 0 && row.Flights == 0, df);

In [ ]:
@time subset(df, :Passengers => ByRow(>(0)), :Seats => ByRow(==(0)), :Flights  => ByRow(==(0)));

Now let us move to another analysis.

We want to get an information about the occupancy of each flight. It can be calculated as ratio of the number of passengers and number of seats time number of flights. The problem is that if there are `0` seats or flights we would be dividing by `0` and get a `NaN` result. We prefer to get a `missing` value in this case. Therefore first define a helper function:

In [ ]:
function get_occupied(passengers, seats, flights)
    if seats == 0 || flights == 0
        return missing
    else
        return passengers / (seats * flights)
    end
end

and now we use it to add a new column to our data frame. Additionally we create three new columns:
1. `:ost`: state of the origin city
2. `:dst`: state of the destination city
3. `:year`: year of flight

In [ ]:
df2 = transform(df,
                [:Passengers, :Seats, :Flights] => ByRow(get_occupied) => :occupied,
                [:Origin_city, :Destination_city] .=> ByRow(x -> last(x, 2)) .=> [:ost, :dst],
                :Fly_date => ByRow(year) => :year)

Notice again that in the computation we used `ByRow` wrapper that instructs DataFrames.jl to apply the function for each row of the passed data.

It is instructive to disect this pipeline step by step. When we said

```julia
[:Passengers, :Seats, :Flights] => ByRow(get_occupied) => :occupied
```

remember that the `get_occupied` function required 3 input arguments. The initial `[:Passengers, :Seats, :Flights]` selector makes sure we pass the required 3 columns, and in the correct order. Finally, having `ByRow(get_occupied) => :occupied` means that our function returns a single output *vector*, and we supply a single name, `:occupied` for it.

In the next statement, we wrote 

```julia
[:Origin_city, :Destination_city] .=> ByRow(x -> last(x, 2)) .=> [:ost, :dst]
```

which looks similar, but has a crucial difference: the `Pair` operator `=>` is being *broadcasted* this time (note the `.` in front of the `=>`)! In simple words, here we will apply our transformation *to each input column*, outputting as well 2 columns (ensured by the second `.=>`). One noteworthy feature of DataFrames.jl is that the so-written components of a data pipeline are valid julia code (and *not* part of a package-specific DSL). For instance, you can evaluate this line in the REPL to investigate the transformation we are building in this step:

```julia
julia> [:Origin_city, :Destination_city] .=> ByRow(x -> last(x, 2)) .=> [:ost, :dst]
2-element Vector{Pair{Symbol, Pair{ByRow{var"#5#6"}, Symbol}}}:
      :Origin_city => (ByRow{var"#5#6"}(var"#5#6"()) => :ost)
 :Destination_city => (ByRow{var"#5#6"}(var"#5#6"()) => :dst)

```

If the central part (the anonymous function with `ByRow` etc) looks confusing, consider this example, which is the essence of what's going on:

```julia
julia> [:x1, :x2] .=> ["op1", "op2"]
2-element Vector{Pair{Symbol, String}}:
 :x1 => "op1"
 :x2 => "op2"
```

This also illustrates that the comma in the output column names array is of crucial importance (i.e. it should be `[:ost, :dst]` and not `[:ost :dst]`). The reason being simply that the julia broadcasting rules would expand the expression into the second dimension, i.e. 

```julia
julia> [:x1, :x2] .=> ["op1" "op2"]
2×2 Matrix{Pair{Symbol, String}}:
 :x1=>"op1"  :x1=>"op2"
 :x2=>"op1"  :x2=>"op2"
```

which is not what we want in this instance.

Here is an alternative way to compute `:occupied` without using `ByRow`, but instead using standard broadcasting:

In [ ]:
get_occupied2(passengers, seats, flights) =
    @. ifelse((seats == 0) || (flights == 0), missing, passengers / (seats * flights))

In [ ]:
df2′ = select(df, [:Passengers, :Seats, :Flights] => get_occupied2 => :occupied);

First we check if both methods produced the same result:

In [ ]:
isequal(df2.occupied, df2′.occupied)

The second thing we can do is compare the performance of both options:

In [ ]:
@benchmark select($df, [:Passengers, :Seats, :Flights] => ByRow(get_occupied) => :occupied)

In [ ]:
@benchmark select($df, [:Passengers, :Seats, :Flights] => get_occupied2 => :occupied)

As you can see DataFrames.jl does a decent job in this case - the option with `ByRow` is a tad faster than the broadcasting approach.

A crucial value of piping with `@chain` is that it composes also with functions outside of DataFrames.jl package.

Here is a simple example in which we will plot the total number of passengers that flew by year:

In [ ]:
@chain df2 begin
    groupby(:year, sort=true)
    combine(:Passengers => sum)
    plot(_.year, _.Passengers_sum ./ 10^6, legend=nothing,
         xlab="year", ylab="passengers (in millions)")
end

I particular note that `_` notation from Chain.jl package allows you to write any valid code you could use on the passed value. In our case `plot` is passed a `DataFrame`, so we e.g. write `_.year` to get a `:year` column from it.

In the following example we will want to find pairs of origin-destination states that have the highest occupancy:

In [ ]:
df3 = @chain df2 begin
    groupby([:ost, :dst])
    combine(:occupied => mean∘skipmissing => :occupied, :Passengers => sum)
    filter!(:occupied => isfinite, _)
    sort!(:occupied, rev=true)
end

In the above code note two things: we use `filter!` and `sort!` functions to process the data frame returned by `combine` in place (which reduces the number of allocations we make).

Additionally note that when we aggregate `:occupied` column in `combine` using `mean∘skipmissing` then for some origin-destination state combinations we get zero observations, which produce `NaN`, so we filter them out using `isfinite` predicate.

Just to make sure what happens when we apply `mean∘skipmissing` to a vector containing only missing values consider the following simple example:

In [ ]:
@chain DataFrame(g=[1,1,2,2], v=[1,2,missing, missing]) begin
    groupby(:g)
    combine(:v => mean∘skipmissing)
end

As a final example let us consider the following case. Assume that you want to query the `df3` data frame by `:ost` and `:dst` columns very often.

The simplest way to do it is to use `subset` as we have already discussed:

In [ ]:
subset(df3, :ost => ByRow(==("NE")), :dst => ByRow(==("AK")))

Now let us use a bit different technique. We `groupby` the `df3` data frame first:

In [ ]:
gdf3 = groupby(df3, [:ost, :dst])

Now we can easiliy query it by group values (here we show such indexing by `Tuple` or `NamedTuple`):

In [ ]:
gdf3[("NE", "AK")]

In [ ]:
gdf3[(ost="NE", dst="AK")]

Note that you can use `keys` on `GroupedDataFrame`:

In [ ]:
keys(gdf3)

In this way we got only the origin-destination state combinations that are actually present in our `GroupedDataFrame`.

The ability to query a `GroupedDataFrame` by grouping columns fast makes it a very useful way to add an index to a data frame.

It is also fast

In [ ]:
@benchmark subset(df3, :ost => ByRow(==("NE")), :dst => ByRow(==("AK")))

In [ ]:
@benchmark gdf3[("NE", "AK")]